In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# /content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles

In [ ]:
import pandas as pd


In [ ]:
def read_model_outfile(outfile_path, model_name):
    df =pd.read_csv(outfile_path)
    df.rename(columns={'prediction': f'{model_name}_preds'}, inplace=True)
    df.rename(columns={'Probabilities': f'{model_name}_probs'}, inplace=True)
    return df

In [ ]:
def find_errors(data:pd.DataFrame, model_name:str,  classes_name:list):
    all_errors = dict()
    for label in classes_name:
        for pred in classes_name:
            if label == pred:
                continue
            errors = data[(data['labels'] == label) & (data[f'{model_name}_preds'] == pred)]
            if errors.shape[0] == 0:
                continue
            all_errors[f"{label}_{pred}"] = errors
    return all_errors

In [ ]:
def check_error_types(error_df):
    print(error_df.groupby("UID").size())

In [ ]:
def marge_model_outfiles(babylm_outfile_df, o_roberta_outfile_df, bb_roberta_outfile_df):

    final_df = pd.DataFrame(babylm_outfile_df)
    final_df['BB_RoBERTa_preds'] = bb_roberta_outfile_df['BB_RoBERTa_preds']
    final_df['BB_RoBERTa_probs'] = bb_roberta_outfile_df['BB_RoBERTa_probs']
    final_df['O_RoBERTa_preds'] = o_roberta_outfile_df['O_RoBERTa_preds']
    final_df['O_RoBERTa_probs'] = o_roberta_outfile_df['O_RoBERTa_probs']

    return final_df

In [ ]:
def compare_models_errors(all_df, model_1, model_2):
    map_list = {"BabyLM": "v20_l4_e9_preds", "O_RoBERTa": "O_RoBERTa_preds", "BB_RoBERTa": "BB_RoBERTa_preds"}
    df = all_df[(all_df['labels'] == all_df[map_list[model_1]]) & (all_df['labels'] != all_df[map_list[model_2]])]
    return df

In [ ]:
def models_compare_labels(all_df, list_correct_model_name, list_wrong_model_name):

    mask_A = all_df[list_correct_model_name].eq(all_df['labels'], axis=0).all(axis=1)
    mask_B = all_df[list_wrong_model_name].ne(all_df['labels'], axis=0).all(axis=1)

    filtered_df = all_df[mask_A & mask_B]

    return filtered_df

### BabyLm

In [ ]:
# df = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles/outfile_v20_l4_e9.csv', 'v20_l4_e9')
df = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles_with_prob/outfile_prob_v20_l4_e9.csv', 'v20_l4_e9')
df.head()

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
0,Those customers examine those commentaries.,G,regular_plural_subject_verb_agreement_1,G,"{'G': 5.1953125, 'PREP': -2.23046875, 'DET': -..."
1,Even Carlos will often happen to die.,G,npi_present_1,G,"{'G': 5.09375, 'PREP': -2.142578125, 'DET': -4..."
2,Ordinary lessons attended by students from dif...,PREP,Preposition_Replacing,G,"{'G': 5.171875, 'PREP': -2.078125, 'DET': -4.4..."
3,Margaret has said Craig discusses himself.,G,principle_A_domain_2,G,"{'G': 5.12109375, 'PREP': -2.26953125, 'DET': ..."
4,Julia skated around that popular cafe.,G,determiner_noun_agreement_with_adjective_1,G,"{'G': 4.89453125, 'PREP': -2.4453125, 'DET': -..."


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]
errors = find_errors(df, 'v20_l4_e9', classes)

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors.keys():
    print(f"{key}: {errors[key].shape}")

G_PREP: (551, 5)
G_SVA: (43, 5)
G_DET: (10, 5)
PREP_G: (186, 5)
PREP_SVA: (1, 5)
SVA_G: (59, 5)
SVA_DET: (1, 5)


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors.keys():
    print(f"{key}:\n {errors[key].head()}\n=============\n")

G_PREP:
                                                  text labels  \
27  I was going to markets and to schools and I wa...      G   
29  So, I think July is the best time to go to the...      G   
32  So would you please reschedule the visit to th...      G   
39  About the activities that I have to choose, I ...      G   
72  Firstly, Palace Hotel has been booked for the ...      G   

                        UID v20_l4_e9_preds  \
27      Preposition_Missing            PREP   
29    Preposition_Replacing            PREP   
32    Preposition_Replacing            PREP   
39    Preposition_Replacing            PREP   
72  Preposition_Unnecessary            PREP   

                                      v20_l4_e9_probs  
27  {'G': 2.626953125, 'PREP': 3.3984375, 'DET': -...  
29  {'G': 2.583984375, 'PREP': 3.431640625, 'DET':...  
32  {'G': 2.5703125, 'PREP': 3.4140625, 'DET': -4....  
39  {'G': 2.623046875, 'PREP': 3.392578125, 'DET':...  
72  {'G': 3.009765625, 'PREP': 3.07421875, 

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors.keys():
    print(f"{key}:\n")
    check_error_types(errors[key])
    print("\n=============\n")

G_PREP:

UID
Preposition_Missing        128
Preposition_Replacing      312
Preposition_Unnecessary    111
dtype: int64


G_SVA:

UID
Preposition_Replacing                         1
Preposition_Unnecessary                       1
animate_subject_passive                       1
animate_subject_trans                         1
causative                                     1
determiner_noun_agreement_2                   1
distractor_agreement_relational_noun          4
distractor_agreement_relative_clause         14
drop_argument                                 1
ellipsis_n_bar_1                              1
ellipsis_n_bar_2                              1
inchoative                                    1
intransitive                                  1
irregular_past_participle_verbs               2
irregular_plural_subject_verb_agreement_1     4
irregular_plural_subject_verb_agreement_2     1
passive_1                                     1
principle_A_domain_1                          1
reg

In [ ]:
errors['G_DET'].head(20)

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
1372,Lawrence had concealed what that cake resembles.,G,wh_vs_that_with_gap,DET,"{'G': -1.421875, 'PREP': -1.71875, 'DET': 8.03..."
2613,Some children question every child that the mo...,G,wh_questions_object_gap,DET,"{'G': -1.41796875, 'PREP': -1.6142578125, 'DET..."
2697,It's himself that Aaron approached.,G,principle_A_reconstruction,DET,"{'G': 1.7109375, 'PREP': -2.291015625, 'DET': ..."
3218,Sarah researched some phenomenon that Jason re...,G,wh_questions_object_gap,DET,"{'G': 1.80078125, 'PREP': -2.748046875, 'DET':..."
3523,Travis should notice this hospital that all mo...,G,wh_questions_subject_gap_long_distance,DET,"{'G': 1.740234375, 'PREP': -2.310546875, 'DET'..."
4207,Some waiter should notice this mirror that Mer...,G,wh_questions_subject_gap_long_distance,DET,"{'G': 0.488525390625, 'PREP': -2.30078125, 'DE..."
4633,Lori thought about the girls that all girls at...,G,wh_questions_subject_gap_long_distance,DET,"{'G': 1.2333984375, 'PREP': -2.69140625, 'DET'..."
5719,Those dancers think about themselves selling t...,G,principle_A_case_2,DET,"{'G': -1.8486328125, 'PREP': -1.1689453125, 'D..."
6005,Banks have thought about themselves confusing ...,G,principle_A_case_2,DET,"{'G': -0.027557373046875, 'PREP': -2.548828125..."
6690,Renee noticed some lady that cashiers praise.,G,wh_questions_object_gap,DET,"{'G': -0.411865234375, 'PREP': -2.46484375, 'D..."


In [ ]:
errors['G_SVA'].head(50)

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
105,A lot of waiters who didn't dislike Katherine ...,G,distractor_agreement_relative_clause,SVA,"{'G': 2.16796875, 'PREP': -2.94140625, 'DET': ..."
274,Sharon froze the ice cream.,G,causative,SVA,"{'G': 0.81298828125, 'PREP': -2.806640625, 'DE..."
343,The Borgias shocks Elaine.,G,regular_plural_subject_verb_agreement_1,SVA,"{'G': -0.74267578125, 'PREP': -2.1171875, 'DET..."
1093,Boys weren't assembling.,G,inchoative,SVA,"{'G': -1.1083984375, 'PREP': -2.1796875, 'DET'..."
1191,The child hunts.,G,irregular_plural_subject_verb_agreement_2,SVA,"{'G': -1.376953125, 'PREP': -1.833984375, 'DET..."
1221,Children wave.,G,irregular_plural_subject_verb_agreement_1,SVA,"{'G': -0.5849609375, 'PREP': -2.072265625, 'DE..."
1329,All adults who hire Bruce ascend some slope.,G,distractor_agreement_relative_clause,SVA,"{'G': 2.0078125, 'PREP': -2.9765625, 'DET': -4..."
2216,Most commentaries about the Balkans will disag...,G,intransitive,SVA,"{'G': 0.8388671875, 'PREP': -2.908203125, 'DET..."
2235,Many pedestrians lifted those movie theaters.,G,determiner_noun_agreement_2,SVA,"{'G': 0.9169921875, 'PREP': -2.787109375, 'DET..."
2254,These senators who didn't respect Valerie aren...,G,distractor_agreement_relative_clause,SVA,"{'G': 2.703125, 'PREP': -2.78125, 'DET': -4.84..."


In [ ]:
errors['SVA_DET'].head()

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
5201,The sketch of these pictures alarm April.,SVA,distractor_agreement_relational_noun,DET,"{'G': -3.63671875, 'PREP': -1.181640625, 'DET'..."


In [ ]:
errors['PREP_SVA'].head()

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
214,The truth of Shopping,PREP,Preposition_Replacing,SVA,"{'G': 0.47705078125, 'PREP': -2.91015625, 'DET..."


In [ ]:
errors['SVA_G'].head(60)

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs
33,Analyses persuades this cousin of Brad to exer...,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 4.53125, 'PREP': -2.787109375, 'DET': -4..."
138,The sister of all cashiers force Andrea's best...,SVA,distractor_agreement_relational_noun,G,"{'G': 5.140625, 'PREP': -2.37890625, 'DET': -4..."
139,The person bores Kevin.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 5.0859375, 'PREP': -2.37890625, 'DET': -..."
149,Candice return to Eva.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 5.16796875, 'PREP': -2.177734375, 'DET':..."
172,The granddaughters of Eva has explained these ...,SVA,distractor_agreement_relational_noun,G,"{'G': 3.86328125, 'PREP': -2.521484375, 'DET':..."
198,A lot of senators who had disturbed Ellen prof...,SVA,distractor_agreement_relative_clause,G,"{'G': 2.9375, 'PREP': -3.0390625, 'DET': -4.45..."
260,Those children respects most waiters.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 4.265625, 'PREP': -2.841796875, 'DET': -..."
311,Grace skate around those libraries.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 4.96875, 'PREP': -2.498046875, 'DET': -4..."
333,The essay about the Great Lakes confuse Brett.,SVA,distractor_agreement_relational_noun,G,"{'G': 4.56640625, 'PREP': -2.71484375, 'DET': ..."
419,Some actresses distract Michael.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 4.86328125, 'PREP': -2.5625, 'DET': -4.7..."


### O_RoBERTa

In [ ]:
# df_O = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles/outfile_o_RoBERTa.csv', 'O_RoBERTa')
df_O = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles_with_prob/outfile_prob_o_RoBERTa.csv', 'O_RoBERTa')
df_O.head()

,text,labels,UID,O_RoBERTa_preds,O_RoBERTa_probs
0,Those customers examine those commentaries.,G,regular_plural_subject_verb_agreement_1,G,"{'G': 7.078125, 'PREP': -2.697265625, 'DET': -..."
1,Even Carlos will often happen to die.,G,npi_present_1,G,"{'G': 7.24609375, 'PREP': -2.16015625, 'DET': ..."
2,Ordinary lessons attended by students from dif...,PREP,Preposition_Replacing,G,"{'G': 6.9375, 'PREP': -1.875, 'DET': -6.285156..."
3,Margaret has said Craig discusses himself.,G,principle_A_domain_2,G,"{'G': 7.203125, 'PREP': -2.46875, 'DET': -5.24..."
4,Julia skated around that popular cafe.,G,determiner_noun_agreement_with_adjective_1,G,"{'G': 7.20703125, 'PREP': -2.451171875, 'DET':..."


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]
errors_O = find_errors(df_O, 'O_RoBERTa', classes)

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_O.keys():
    print(f"{key}: {errors_O[key].shape}")

G_PREP: (209, 5)
G_SVA: (4, 5)
G_DET: (1, 5)
PREP_G: (336, 5)
PREP_SVA: (1, 5)
SVA_G: (9, 5)


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_O.keys():
    print(f"{key}:\n {errors_O[key].head()}\n=============\n")

G_PREP:
                                                   text labels  \
27   I was going to markets and to schools and I wa...      G   
113  Furthermore, as mathematics is the main lesson...      G   
134  On behalf of all the students, I thank you for...      G   
140  I attended your festival and I would like to g...      G   
182  People have invented such things as TV, radio,...      G   

                       UID O_RoBERTa_preds  \
27     Preposition_Missing            PREP   
113    Preposition_Missing            PREP   
134  Preposition_Replacing            PREP   
140  Preposition_Replacing            PREP   
182  Preposition_Replacing            PREP   

                                       O_RoBERTa_probs  
27   {'G': 2.431640625, 'PREP': 4.90625, 'DET': -6....  
113  {'G': 2.0390625, 'PREP': 5.30859375, 'DET': -6...  
134  {'G': 3.513671875, 'PREP': 3.623046875, 'DET':...  
140  {'G': 1.6171875, 'PREP': 5.56640625, 'DET': -6...  
182  {'G': 2.265625, 'PREP': 5.140625,

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_O.keys():
    print(f"{key}:\n")
    check_error_types(errors_O[key])
    print("\n=============\n")

G_PREP:

UID
Preposition_Missing         43
Preposition_Replacing      134
Preposition_Unnecessary     32
dtype: int64


G_SVA:

UID
determiner_noun_agreement_irregular_1      1
determiner_noun_agreement_with_adj_2       1
only_npi_scope                             1
regular_plural_subject_verb_agreement_1    1
dtype: int64


G_DET:

UID
ellipsis_n_bar_2    1
dtype: int64


PREP_G:

UID
Preposition_Missing        100
Preposition_Replacing      174
Preposition_Unnecessary     62
dtype: int64


PREP_SVA:

UID
Preposition_Replacing    1
dtype: int64


SVA_G:

UID
irregular_plural_subject_verb_agreement_1    3
irregular_plural_subject_verb_agreement_2    2
regular_plural_subject_verb_agreement_1      2
regular_plural_subject_verb_agreement_2      2
dtype: int64




In [ ]:
errors_O['G_SVA'].head()

,text,labels,UID,O_RoBERTa_preds,O_RoBERTa_probs
164,Only steak that a lot of guests notice ever em...,G,only_npi_scope,SVA,"{'G': 2.525390625, 'PREP': -3.08984375, 'DET':..."
3189,Most ladies confuses that person.,G,determiner_noun_agreement_irregular_1,SVA,"{'G': 0.076904296875, 'PREP': -3.162109375, 'D..."
4358,Some actresses distracts Michael.,G,regular_plural_subject_verb_agreement_1,SVA,"{'G': -0.52099609375, 'PREP': -3.11328125, 'DE..."
6833,All drivers confuses those ill actresses.,G,determiner_noun_agreement_with_adj_2,SVA,"{'G': -0.84814453125, 'PREP': -3.22265625, 'DE..."


In [ ]:
errors_O['G_DET'].head()

,text,labels,UID,O_RoBERTa_preds,O_RoBERTa_probs
2133,Matt can reveal one school and Lawrence can re...,G,ellipsis_n_bar_2,DET,"{'G': 3.20703125, 'PREP': -3.44921875, 'DET': ..."


In [ ]:
errors_O['PREP_SVA'].head()

,text,labels,UID,O_RoBERTa_preds,O_RoBERTa_probs
214,The truth of Shopping,PREP,Preposition_Replacing,SVA,"{'G': -1.3173828125, 'PREP': -1.53125, 'DET': ..."


In [ ]:
errors_O['SVA_G'].head(10)

,text,labels,UID,O_RoBERTa_preds,O_RoBERTa_probs
139,The person bores Kevin.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 6.98046875, 'PREP': -2.77734375, 'DET': ..."
419,Some actresses distract Michael.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 6.640625, 'PREP': -2.888671875, 'DET': -..."
803,A lot of alumni bother some hospital.,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 6.91796875, 'PREP': -2.712890625, 'DET':..."
1231,The noses bothers Alicia.,SVA,regular_plural_subject_verb_agreement_2,G,"{'G': 4.265625, 'PREP': -2.76953125, 'DET': -5..."
1542,The oases hurts this doctor.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 6.828125, 'PREP': -2.80859375, 'DET': -5..."
1621,A lot of men insults Wayne.,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 4.73828125, 'PREP': -2.65234375, 'DET': ..."
6032,The glasses bothers those senators.,SVA,regular_plural_subject_verb_agreement_2,G,"{'G': 6.5625, 'PREP': -2.84375, 'DET': -5.4179..."
6256,The Borgias shock Elaine.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 6.78125, 'PREP': -2.888671875, 'DET': -5..."
6977,Those men obliges every bank to cooperate.,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 7.0703125, 'PREP': -2.42578125, 'DET': -..."


### BB_RoBERTa

In [ ]:
# df_BB = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles/outfile_bb_RoBERTa.csv', 'BB_RoBERTa')
df_BB = read_model_outfile('/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/outfiles_with_prob/outfile_prob_bb_RoBERTa.csv', 'BB_RoBERTa')
df_BB.head()

,text,labels,UID,BB_RoBERTa_preds,BB_RoBERTa_probs
0,Those customers examine those commentaries.,G,regular_plural_subject_verb_agreement_1,G,"{'G': 6.7890625, 'PREP': -2.380859375, 'DET': ..."
1,Even Carlos will often happen to die.,G,npi_present_1,G,"{'G': 6.71875, 'PREP': -1.349609375, 'DET': -4..."
2,Ordinary lessons attended by students from dif...,PREP,Preposition_Replacing,G,"{'G': 6.84375, 'PREP': -2.142578125, 'DET': -4..."
3,Margaret has said Craig discusses himself.,G,principle_A_domain_2,G,"{'G': 6.86328125, 'PREP': -2.03125, 'DET': -4...."
4,Julia skated around that popular cafe.,G,determiner_noun_agreement_with_adjective_1,G,"{'G': 6.765625, 'PREP': -2.310546875, 'DET': -..."


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]
errors_BB = find_errors(df_BB, 'BB_RoBERTa', classes)

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_BB.keys():
    print(f"{key}: {errors_BB[key].shape}")

G_PREP: (529, 5)
G_SVA: (13, 5)
G_DET: (1, 5)
PREP_G: (258, 5)
PREP_SVA: (4, 5)
SVA_G: (15, 5)


In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_BB.keys():
    print(f"{key}:\n {errors_BB[key].head()}\n=============\n")

G_PREP:
                                                  text labels  \
27  I was going to markets and to schools and I wa...      G   
29  So, I think July is the best time to go to the...      G   
32  So would you please reschedule the visit to th...      G   
39  About the activities that I have to choose, I ...      G   
77  I am writing to you because there were deviati...      G   

                      UID BB_RoBERTa_preds  \
27    Preposition_Missing             PREP   
29  Preposition_Replacing             PREP   
32  Preposition_Replacing             PREP   
39  Preposition_Replacing             PREP   
77  Preposition_Replacing             PREP   

                                     BB_RoBERTa_probs  
27  {'G': 3.884765625, 'PREP': 4.62890625, 'DET': ...  
29  {'G': 3.7421875, 'PREP': 4.671875, 'DET': -5.3...  
32  {'G': 3.96875, 'PREP': 4.546875, 'DET': -5.453...  
39  {'G': 3.513671875, 'PREP': 4.77734375, 'DET': ...  
77  {'G': 4.12890625, 'PREP': 4.44140625, 'DET': 

In [ ]:
classes = ["G", "PREP", "SVA", "DET"]

for key in errors_BB.keys():
    print(f"{key}:\n")
    check_error_types(errors_BB[key])
    print("\n=============\n")

G_PREP:

UID
Preposition_Missing        117
Preposition_Replacing      305
Preposition_Unnecessary    107
dtype: int64


G_SVA:

UID
Preposition_Replacing                             1
Preposition_Unnecessary                           2
determiner_noun_agreement_with_adj_irregular_2    1
distractor_agreement_relative_clause              2
intransitive                                      1
irregular_plural_subject_verb_agreement_1         1
irregular_plural_subject_verb_agreement_2         1
only_npi_scope                                    1
regular_plural_subject_verb_agreement_1           2
regular_plural_subject_verb_agreement_2           1
dtype: int64


G_DET:

UID
wh_questions_object_gap    1
dtype: int64


PREP_G:

UID
Preposition_Missing         66
Preposition_Replacing      130
Preposition_Unnecessary     62
dtype: int64


PREP_SVA:

UID
Preposition_Missing        1
Preposition_Replacing      1
Preposition_Unnecessary    2
dtype: int64


SVA_G:

UID
distractor_agreement_relat

In [ ]:
errors_BB['G_SVA'].head(15)

,text,labels,UID,BB_RoBERTa_preds,BB_RoBERTa_probs
164,Only steak that a lot of guests notice ever em...,G,only_npi_scope,SVA,"{'G': 1.73046875, 'PREP': -3.76171875, 'DET': ..."
343,The Borgias shocks Elaine.,G,regular_plural_subject_verb_agreement_1,SVA,"{'G': -0.288818359375, 'PREP': -3.259765625, '..."
1191,The child hunts.,G,irregular_plural_subject_verb_agreement_2,SVA,"{'G': -0.654296875, 'PREP': -3.453125, 'DET': ..."
1782,.,G,Preposition_Unnecessary,SVA,"{'G': 0.58984375, 'PREP': -2.744140625, 'DET':..."
3057,The truth about Shopping,G,Preposition_Replacing,SVA,"{'G': 0.313720703125, 'PREP': -3.494140625, 'D..."
3257,First tell your parents.,G,Preposition_Unnecessary,SVA,"{'G': -1.9970703125, 'PREP': -2.4140625, 'DET'..."
3576,A lot of alumni bothers some hospital.,G,irregular_plural_subject_verb_agreement_1,SVA,"{'G': -0.033935546875, 'PREP': -3.361328125, '..."
4358,Some actresses distracts Michael.,G,regular_plural_subject_verb_agreement_1,SVA,"{'G': 0.56689453125, 'PREP': -3.306640625, 'DE..."
5016,Many brothers of Colleen testified.,G,intransitive,SVA,"{'G': -1.2158203125, 'PREP': -3.240234375, 'DE..."
5548,That dancer who praised these dancers doubts a...,G,distractor_agreement_relative_clause,SVA,"{'G': 1.1640625, 'PREP': -3.65234375, 'DET': -..."


In [ ]:
errors_BB['G_DET'].head()

,text,labels,UID,BB_RoBERTa_preds,BB_RoBERTa_probs
1597,Irene knew senators that Pamela hugs.,G,wh_questions_object_gap,DET,"{'G': -0.75830078125, 'PREP': -2.568359375, 'D..."


In [ ]:
errors_BB['PREP_SVA'].head()

,text,labels,UID,BB_RoBERTa_preds,BB_RoBERTa_probs
214,The truth of Shopping,PREP,Preposition_Replacing,SVA,"{'G': 0.09027099609375, 'PREP': -2.626953125, ..."
552,First tell to your parents.,PREP,Preposition_Unnecessary,SVA,"{'G': -1.0830078125, 'PREP': -3.099609375, 'DE..."
3907,from.,PREP,Preposition_Unnecessary,SVA,"{'G': 2.193359375, 'PREP': -2.25, 'DET': -5.72..."
4066,A lot of money is involved in research to stop...,PREP,Preposition_Missing,SVA,"{'G': -1.52734375, 'PREP': -2.587890625, 'DET'..."


In [ ]:
errors_BB['SVA_G'].head(20)

,text,labels,UID,BB_RoBERTa_preds,BB_RoBERTa_probs
139,The person bores Kevin.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 6.0, 'PREP': -2.921875, 'DET': -5.296875..."
419,Some actresses distract Michael.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 6.65625, 'PREP': -2.630859375, 'DET': -4..."
803,A lot of alumni bother some hospital.,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 6.68359375, 'PREP': -2.599609375, 'DET':..."
1542,The oases hurts this doctor.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 4.7109375, 'PREP': -3.23828125, 'DET': -..."
1707,Governments that praise Melissa commissions th...,SVA,distractor_agreement_relative_clause,G,"{'G': 5.0546875, 'PREP': -2.623046875, 'DET': ..."
1838,Andrea tour rivers.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 6.44921875, 'PREP': -2.6875, 'DET': -4.9..."
1925,Schools criticizes these glaciers.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 5.86328125, 'PREP': -3.041015625, 'DET':..."
2589,Senators listens to Janet.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 5.74609375, 'PREP': -3.111328125, 'DET':..."
3725,The commentary about current events annoy Kirs...,SVA,distractor_agreement_relational_noun,G,"{'G': 3.48046875, 'PREP': -3.53125, 'DET': -5...."
5635,The shawls warps.,SVA,regular_plural_subject_verb_agreement_2,G,"{'G': 5.80859375, 'PREP': -3.19140625, 'DET': ..."


### All

In [ ]:
all_outfiles = marge_model_outfiles(df, df_O, df_BB)
all_outfiles.head()

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
0,Those customers examine those commentaries.,G,regular_plural_subject_verb_agreement_1,G,"{'G': 5.1953125, 'PREP': -2.23046875, 'DET': -...",G,"{'G': 6.7890625, 'PREP': -2.380859375, 'DET': ...",G,"{'G': 7.078125, 'PREP': -2.697265625, 'DET': -..."
1,Even Carlos will often happen to die.,G,npi_present_1,G,"{'G': 5.09375, 'PREP': -2.142578125, 'DET': -4...",G,"{'G': 6.71875, 'PREP': -1.349609375, 'DET': -4...",G,"{'G': 7.24609375, 'PREP': -2.16015625, 'DET': ..."
2,Ordinary lessons attended by students from dif...,PREP,Preposition_Replacing,G,"{'G': 5.171875, 'PREP': -2.078125, 'DET': -4.4...",G,"{'G': 6.84375, 'PREP': -2.142578125, 'DET': -4...",G,"{'G': 6.9375, 'PREP': -1.875, 'DET': -6.285156..."
3,Margaret has said Craig discusses himself.,G,principle_A_domain_2,G,"{'G': 5.12109375, 'PREP': -2.26953125, 'DET': ...",G,"{'G': 6.86328125, 'PREP': -2.03125, 'DET': -4....",G,"{'G': 7.203125, 'PREP': -2.46875, 'DET': -5.24..."
4,Julia skated around that popular cafe.,G,determiner_noun_agreement_with_adjective_1,G,"{'G': 4.89453125, 'PREP': -2.4453125, 'DET': -...",G,"{'G': 6.765625, 'PREP': -2.310546875, 'DET': -...",G,"{'G': 7.20703125, 'PREP': -2.451171875, 'DET':..."


In [ ]:
list_c = []
list_w = ['v20_l4_e9_preds','BB_RoBERTa_preds', 'O_RoBERTa_preds']
all_wrongs = models_compare_labels(all_outfiles, list_c, list_w)
print(all_wrongs.shape)
all_wrongs.head()

(258, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
2,Ordinary lessons attended by students from dif...,PREP,Preposition_Replacing,G,"{'G': 5.171875, 'PREP': -2.078125, 'DET': -4.4...",G,"{'G': 6.84375, 'PREP': -2.142578125, 'DET': -4...",G,"{'G': 6.9375, 'PREP': -1.875, 'DET': -6.285156..."
27,I was going to markets and to schools and I wa...,G,Preposition_Missing,PREP,"{'G': 2.626953125, 'PREP': 3.3984375, 'DET': -...",PREP,"{'G': 3.884765625, 'PREP': 4.62890625, 'DET': ...",PREP,"{'G': 2.431640625, 'PREP': 4.90625, 'DET': -6...."
113,"Furthermore, as mathematics is the main lesson...",G,Preposition_Missing,PREP,"{'G': 2.443359375, 'PREP': 3.5, 'DET': -4.4609...",PREP,"{'G': 3.345703125, 'PREP': 4.80078125, 'DET': ...",PREP,"{'G': 2.0390625, 'PREP': 5.30859375, 'DET': -6..."
127,You advertised for my perfect evening out.,PREP,Preposition_Unnecessary,G,"{'G': 3.46484375, 'PREP': 2.197265625, 'DET': ...",G,"{'G': 4.5703125, 'PREP': 3.765625, 'DET': -5.4...",G,"{'G': 4.5546875, 'PREP': 1.6748046875, 'DET': ..."
130,"I felt like a fool, and all the audience felt ...",PREP,Preposition_Replacing,G,"{'G': 4.84375, 'PREP': -0.56201171875, 'DET': ...",G,"{'G': 5.49609375, 'PREP': 2.208984375, 'DET': ...",G,"{'G': 6.30078125, 'PREP': -0.07666015625, 'DET..."


In [ ]:
all_wrongs[(all_wrongs['labels'] == 'SVA')].head(10)

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
139,The person bores Kevin.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 5.0859375, 'PREP': -2.37890625, 'DET': -...",G,"{'G': 6.0, 'PREP': -2.921875, 'DET': -5.296875...",G,"{'G': 6.98046875, 'PREP': -2.77734375, 'DET': ..."
419,Some actresses distract Michael.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 4.86328125, 'PREP': -2.5625, 'DET': -4.7...",G,"{'G': 6.65625, 'PREP': -2.630859375, 'DET': -4...",G,"{'G': 6.640625, 'PREP': -2.888671875, 'DET': -..."
803,A lot of alumni bother some hospital.,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 5.09375, 'PREP': -2.42578125, 'DET': -4....",G,"{'G': 6.68359375, 'PREP': -2.599609375, 'DET':...",G,"{'G': 6.91796875, 'PREP': -2.712890625, 'DET':..."
1542,The oases hurts this doctor.,SVA,irregular_plural_subject_verb_agreement_2,G,"{'G': 3.017578125, 'PREP': -3.171875, 'DET': -...",G,"{'G': 4.7109375, 'PREP': -3.23828125, 'DET': -...",G,"{'G': 6.828125, 'PREP': -2.80859375, 'DET': -5..."
6256,The Borgias shock Elaine.,SVA,regular_plural_subject_verb_agreement_1,G,"{'G': 5.078125, 'PREP': -2.431640625, 'DET': -...",G,"{'G': 6.640625, 'PREP': -2.638671875, 'DET': -...",G,"{'G': 6.78125, 'PREP': -2.888671875, 'DET': -5..."


In [ ]:
all_wrongs[(all_wrongs['labels'] == 'G') &
           (all_wrongs['v20_l4_e9_preds'] != 'PREP') &
           (all_wrongs['BB_RoBERTa_preds']!= 'PREP') &
           (all_wrongs['O_RoBERTa_preds'] != 'PREP')].head(10)

,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
4358,Some actresses distracts Michael.,G,regular_plural_subject_verb_agreement_1,SVA,"{'G': -1.3095703125, 'PREP': -1.90234375, 'DET...",SVA,"{'G': 0.56689453125, 'PREP': -3.306640625, 'DE...",SVA,"{'G': -0.52099609375, 'PREP': -3.11328125, 'DE..."


In [ ]:
baby_vs_o_roberta = compare_models_errors(all_outfiles, "BabyLM", "O_RoBERTa")
print(baby_vs_o_roberta.shape)
baby_vs_o_roberta.head()

(265, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
5,Modern technology is used all the time at home...,PREP,Preposition_Missing,PREP,"{'G': 2.787109375, 'PREP': 3.28125, 'DET': -4....",PREP,"{'G': 3.373046875, 'PREP': 4.8046875, 'DET': -...",G,"{'G': 4.48046875, 'PREP': 1.8994140625, 'DET':..."
12,"I can play games, listen to music, and I can d...",PREP,Preposition_Replacing,PREP,"{'G': 2.314453125, 'PREP': 3.546875, 'DET': -4...",PREP,"{'G': 3.251953125, 'PREP': 4.80859375, 'DET': ...",G,"{'G': 4.53515625, 'PREP': 1.8603515625, 'DET':..."
45,"In my opinion, in a hundred of years we'll wea...",PREP,Preposition_Unnecessary,PREP,"{'G': 2.455078125, 'PREP': 3.49609375, 'DET': ...",PREP,"{'G': 3.34765625, 'PREP': 4.80078125, 'DET': -...",G,"{'G': 4.18359375, 'PREP': 2.3359375, 'DET': -6..."
69,"As if that wasn't enough, after the show, I to...",PREP,Preposition_Replacing,PREP,"{'G': 2.66015625, 'PREP': 3.369140625, 'DET': ...",PREP,"{'G': 3.837890625, 'PREP': 4.64453125, 'DET': ...",G,"{'G': 4.9453125, 'PREP': 1.5107421875, 'DET': ..."
99,The reason for this is that I have difficulty ...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.521484375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.345703125, 'PREP': 4.80078125, 'DET': ...",G,"{'G': 4.94140625, 'PREP': 1.4755859375, 'DET':..."


In [ ]:
o_roberta_vs_baby = compare_models_errors(all_outfiles, "O_RoBERTa", "BabyLM")
print(o_roberta_vs_baby.shape)
o_roberta_vs_baby.head()

(556, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
29,"So, I think July is the best time to go to the...",G,Preposition_Replacing,PREP,"{'G': 2.583984375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.7421875, 'PREP': 4.671875, 'DET': -5.3...",G,"{'G': 5.828125, 'PREP': 0.5107421875, 'DET': -..."
32,So would you please reschedule the visit to th...,G,Preposition_Replacing,PREP,"{'G': 2.5703125, 'PREP': 3.4140625, 'DET': -4....",PREP,"{'G': 3.96875, 'PREP': 4.546875, 'DET': -5.453...",G,"{'G': 4.4921875, 'PREP': 1.83203125, 'DET': -6..."
33,Analyses persuades this cousin of Brad to exer...,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 4.53125, 'PREP': -2.787109375, 'DET': -4...",SVA,"{'G': 2.814453125, 'PREP': -3.572265625, 'DET'...",SVA,"{'G': 0.0089874267578125, 'PREP': -3.08203125,..."
39,"About the activities that I have to choose, I ...",G,Preposition_Replacing,PREP,"{'G': 2.623046875, 'PREP': 3.392578125, 'DET':...",PREP,"{'G': 3.513671875, 'PREP': 4.77734375, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 1.884765625, 'DET': ..."
72,"Firstly, Palace Hotel has been booked for the ...",G,Preposition_Unnecessary,PREP,"{'G': 3.009765625, 'PREP': 3.07421875, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 3.984375, 'DET': -5....",G,"{'G': 5.1953125, 'PREP': 1.287109375, 'DET': -..."


In [ ]:
baby_vs_bb_roberta = compare_models_errors(all_outfiles, "BabyLM", "BB_RoBERTa")
print(baby_vs_bb_roberta.shape)
baby_vs_bb_roberta.head()

(171, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
7,I thought it was a good idea to tell to Pat be...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.671875, 'PREP': 3.353515625, 'DET': -4...",G,"{'G': 4.38671875, 'PREP': 4.2421875, 'DET': -5...",PREP,"{'G': 0.51611328125, 'PREP': 5.640625, 'DET': ..."
164,Only steak that a lot of guests notice ever em...,G,only_npi_scope,G,"{'G': 4.74609375, 'PREP': -2.669921875, 'DET':...",SVA,"{'G': 1.73046875, 'PREP': -3.76171875, 'DET': ...",SVA,"{'G': 2.525390625, 'PREP': -3.08984375, 'DET':..."
278,I am replying to your letter about the first p...,PREP,Preposition_Replacing,PREP,"{'G': 2.642578125, 'PREP': 3.388671875, 'DET':...",G,"{'G': 4.6640625, 'PREP': 3.814453125, 'DET': -...",G,"{'G': 4.9921875, 'PREP': 1.44921875, 'DET': -6..."
292,"Lastly, I would like to learn that what kinds ...",PREP,Preposition_Unnecessary,PREP,"{'G': 2.705078125, 'PREP': 3.302734375, 'DET':...",G,"{'G': 5.03125, 'PREP': 3.068359375, 'DET': -5....",PREP,"{'G': 2.125, 'PREP': 5.265625, 'DET': -6.57812..."
331,"For August, I have already made another appoin...",G,Preposition_Replacing,G,"{'G': 3.16015625, 'PREP': 2.853515625, 'DET': ...",PREP,"{'G': 4.21484375, 'PREP': 4.359375, 'DET': -5....",G,"{'G': 5.53515625, 'PREP': 0.90283203125, 'DET'..."


In [ ]:
bb_roberta_vs_baby = compare_models_errors(all_outfiles, "BB_RoBERTa", "BabyLM")
print(bb_roberta_vs_baby.shape)
bb_roberta_vs_baby.head()

(202, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
33,Analyses persuades this cousin of Brad to exer...,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 4.53125, 'PREP': -2.787109375, 'DET': -4...",SVA,"{'G': 2.814453125, 'PREP': -3.572265625, 'DET'...",SVA,"{'G': 0.0089874267578125, 'PREP': -3.08203125,..."
72,"Firstly, Palace Hotel has been booked for the ...",G,Preposition_Unnecessary,PREP,"{'G': 3.009765625, 'PREP': 3.07421875, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 3.984375, 'DET': -5....",G,"{'G': 5.1953125, 'PREP': 1.287109375, 'DET': -..."
105,A lot of waiters who didn't dislike Katherine ...,G,distractor_agreement_relative_clause,SVA,"{'G': 2.16796875, 'PREP': -2.94140625, 'DET': ...",G,"{'G': 6.7265625, 'PREP': -2.427734375, 'DET': ...",G,"{'G': 7.1796875, 'PREP': -2.447265625, 'DET': ..."
132,"Now, I am writing to you to ask for some detai...",G,Preposition_Missing,PREP,"{'G': 2.732421875, 'PREP': 3.30078125, 'DET': ...",G,"{'G': 4.3984375, 'PREP': 4.19140625, 'DET': -5...",G,"{'G': 5.328125, 'PREP': 1.16015625, 'DET': -6...."
138,The sister of all cashiers force Andrea's best...,SVA,distractor_agreement_relational_noun,G,"{'G': 5.140625, 'PREP': -2.37890625, 'DET': -4...",SVA,"{'G': -0.048095703125, 'PREP': -2.521484375, '...",SVA,"{'G': -2.05078125, 'PREP': -3.12109375, 'DET':..."


In [ ]:
bb_roberta_vs_o_roberta = compare_models_errors(all_outfiles, "BB_RoBERTa", "O_RoBERTa")
print(bb_roberta_vs_o_roberta.shape)
bb_roberta_vs_o_roberta.head()

(230, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
5,Modern technology is used all the time at home...,PREP,Preposition_Missing,PREP,"{'G': 2.787109375, 'PREP': 3.28125, 'DET': -4....",PREP,"{'G': 3.373046875, 'PREP': 4.8046875, 'DET': -...",G,"{'G': 4.48046875, 'PREP': 1.8994140625, 'DET':..."
12,"I can play games, listen to music, and I can d...",PREP,Preposition_Replacing,PREP,"{'G': 2.314453125, 'PREP': 3.546875, 'DET': -4...",PREP,"{'G': 3.251953125, 'PREP': 4.80859375, 'DET': ...",G,"{'G': 4.53515625, 'PREP': 1.8603515625, 'DET':..."
45,"In my opinion, in a hundred of years we'll wea...",PREP,Preposition_Unnecessary,PREP,"{'G': 2.455078125, 'PREP': 3.49609375, 'DET': ...",PREP,"{'G': 3.34765625, 'PREP': 4.80078125, 'DET': -...",G,"{'G': 4.18359375, 'PREP': 2.3359375, 'DET': -6..."
69,"As if that wasn't enough, after the show, I to...",PREP,Preposition_Replacing,PREP,"{'G': 2.66015625, 'PREP': 3.369140625, 'DET': ...",PREP,"{'G': 3.837890625, 'PREP': 4.64453125, 'DET': ...",G,"{'G': 4.9453125, 'PREP': 1.5107421875, 'DET': ..."
99,The reason for this is that I have difficulty ...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.521484375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.345703125, 'PREP': 4.80078125, 'DET': ...",G,"{'G': 4.94140625, 'PREP': 1.4755859375, 'DET':..."


In [ ]:
o_roberta_vs_bb_roberta = compare_models_errors(all_outfiles, "O_RoBERTa", "BB_RoBERTa")
print(o_roberta_vs_bb_roberta.shape)
o_roberta_vs_bb_roberta.head()

(490, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
7,I thought it was a good idea to tell to Pat be...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.671875, 'PREP': 3.353515625, 'DET': -4...",G,"{'G': 4.38671875, 'PREP': 4.2421875, 'DET': -5...",PREP,"{'G': 0.51611328125, 'PREP': 5.640625, 'DET': ..."
29,"So, I think July is the best time to go to the...",G,Preposition_Replacing,PREP,"{'G': 2.583984375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.7421875, 'PREP': 4.671875, 'DET': -5.3...",G,"{'G': 5.828125, 'PREP': 0.5107421875, 'DET': -..."
32,So would you please reschedule the visit to th...,G,Preposition_Replacing,PREP,"{'G': 2.5703125, 'PREP': 3.4140625, 'DET': -4....",PREP,"{'G': 3.96875, 'PREP': 4.546875, 'DET': -5.453...",G,"{'G': 4.4921875, 'PREP': 1.83203125, 'DET': -6..."
39,"About the activities that I have to choose, I ...",G,Preposition_Replacing,PREP,"{'G': 2.623046875, 'PREP': 3.392578125, 'DET':...",PREP,"{'G': 3.513671875, 'PREP': 4.77734375, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 1.884765625, 'DET': ..."
77,I am writing to you because there were deviati...,G,Preposition_Replacing,PREP,"{'G': 2.62890625, 'PREP': 3.390625, 'DET': -4....",PREP,"{'G': 4.12890625, 'PREP': 4.44140625, 'DET': -...",G,"{'G': 6.0546875, 'PREP': 0.3037109375, 'DET': ..."


In [ ]:
list_c = ['v20_l4_e9_preds']
list_w = ['BB_RoBERTa_preds', 'O_RoBERTa_preds']
baby_vs_O_and_BB = models_compare_labels(all_outfiles, list_c, list_w)
print(baby_vs_O_and_BB.shape)
baby_vs_O_and_BB.head()

(72, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
164,Only steak that a lot of guests notice ever em...,G,only_npi_scope,G,"{'G': 4.74609375, 'PREP': -2.669921875, 'DET':...",SVA,"{'G': 1.73046875, 'PREP': -3.76171875, 'DET': ...",SVA,"{'G': 2.525390625, 'PREP': -3.08984375, 'DET':..."
278,I am replying to your letter about the first p...,PREP,Preposition_Replacing,PREP,"{'G': 2.642578125, 'PREP': 3.388671875, 'DET':...",G,"{'G': 4.6640625, 'PREP': 3.814453125, 'DET': -...",G,"{'G': 4.9921875, 'PREP': 1.44921875, 'DET': -6..."
429,"Of course, you should give a refund of the mon...",PREP,Preposition_Replacing,PREP,"{'G': 2.60546875, 'PREP': 3.40234375, 'DET': -...",G,"{'G': 4.35546875, 'PREP': 4.22265625, 'DET': -...",G,"{'G': 4.578125, 'PREP': 1.7421875, 'DET': -6.5..."
454,"So, you run to the bank but have to wait for t...",PREP,Preposition_Replacing,PREP,"{'G': 2.662109375, 'PREP': 3.3671875, 'DET': -...",G,"{'G': 4.828125, 'PREP': 3.26953125, 'DET': -5....",G,"{'G': 5.390625, 'PREP': 1.03515625, 'DET': -6...."
482,This is because people won't give up the great...,PREP,Preposition_Replacing,PREP,"{'G': 2.8984375, 'PREP': 3.126953125, 'DET': -...",G,"{'G': 4.609375, 'PREP': 3.912109375, 'DET': -5...",G,"{'G': 4.8671875, 'PREP': 1.537109375, 'DET': -..."


In [ ]:
list_c = ['BB_RoBERTa_preds']
list_w = ['v20_l4_e9_preds','O_RoBERTa_preds']
BB_vs_baby_and_O = models_compare_labels(all_outfiles, list_c, list_w)
print(BB_vs_baby_and_O.shape)
BB_vs_baby_and_O.head()

(37, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
489,"As requested, the aim of this report is to ass...",G,Preposition_Replacing,PREP,"{'G': 2.884765625, 'PREP': 3.103515625, 'DET':...",G,"{'G': 4.80859375, 'PREP': 3.595703125, 'DET': ...",PREP,"{'G': 1.4267578125, 'PREP': 5.640625, 'DET': -..."
720,"When I bought my tickets, I noticed I didn't g...",G,Preposition_Replacing,PREP,"{'G': 2.73046875, 'PREP': 3.3125, 'DET': -4.54...",G,"{'G': 4.4296875, 'PREP': 4.1015625, 'DET': -5....",PREP,"{'G': 1.947265625, 'PREP': 5.40625, 'DET': -6...."
799,My second complaint is about some the concert ...,PREP,Preposition_Missing,G,"{'G': 4.8203125, 'PREP': -0.82666015625, 'DET'...",PREP,"{'G': 3.923828125, 'PREP': 4.55078125, 'DET': ...",G,"{'G': 5.5, 'PREP': 0.91943359375, 'DET': -6.75..."
1041,To my personal opinion one of the most importa...,PREP,Preposition_Replacing,G,"{'G': 3.115234375, 'PREP': 2.904296875, 'DET':...",PREP,"{'G': 3.37109375, 'PREP': 4.796875, 'DET': -5....",G,"{'G': 4.921875, 'PREP': 1.51171875, 'DET': -6...."
1232,The idea of going to the science museum is fan...,G,Preposition_Replacing,PREP,"{'G': 2.708984375, 'PREP': 3.34765625, 'DET': ...",G,"{'G': 4.484375, 'PREP': 4.1328125, 'DET': -5.6...",PREP,"{'G': 2.193359375, 'PREP': 5.2109375, 'DET': -..."


In [ ]:
list_c = ['O_RoBERTa_preds']
list_w = ['v20_l4_e9_preds','BB_RoBERTa_preds']
O_vs_baby_and_BB = models_compare_labels(all_outfiles, list_c, list_w)
print(O_vs_baby_and_BB.shape)
O_vs_baby_and_BB.head()

(391, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
29,"So, I think July is the best time to go to the...",G,Preposition_Replacing,PREP,"{'G': 2.583984375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.7421875, 'PREP': 4.671875, 'DET': -5.3...",G,"{'G': 5.828125, 'PREP': 0.5107421875, 'DET': -..."
32,So would you please reschedule the visit to th...,G,Preposition_Replacing,PREP,"{'G': 2.5703125, 'PREP': 3.4140625, 'DET': -4....",PREP,"{'G': 3.96875, 'PREP': 4.546875, 'DET': -5.453...",G,"{'G': 4.4921875, 'PREP': 1.83203125, 'DET': -6..."
39,"About the activities that I have to choose, I ...",G,Preposition_Replacing,PREP,"{'G': 2.623046875, 'PREP': 3.392578125, 'DET':...",PREP,"{'G': 3.513671875, 'PREP': 4.77734375, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 1.884765625, 'DET': ..."
77,I am writing to you because there were deviati...,G,Preposition_Replacing,PREP,"{'G': 2.62890625, 'PREP': 3.390625, 'DET': -4....",PREP,"{'G': 4.12890625, 'PREP': 4.44140625, 'DET': -...",G,"{'G': 6.0546875, 'PREP': 0.3037109375, 'DET': ..."
87,Then we thought about going swimming to talk a...,G,Preposition_Unnecessary,PREP,"{'G': 2.83203125, 'PREP': 3.15625, 'DET': -4.2...",PREP,"{'G': 4.03515625, 'PREP': 4.4921875, 'DET': -5...",G,"{'G': 5.02734375, 'PREP': 1.28515625, 'DET': -..."


In [ ]:
list_c = ['v20_l4_e9_preds','BB_RoBERTa_preds']
list_w = ['O_RoBERTa_preds']
baby_and_BB_vs_O = models_compare_labels(all_outfiles, list_c, list_w)
print(baby_and_BB_vs_O.shape)
baby_and_BB_vs_O.head()

(193, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
5,Modern technology is used all the time at home...,PREP,Preposition_Missing,PREP,"{'G': 2.787109375, 'PREP': 3.28125, 'DET': -4....",PREP,"{'G': 3.373046875, 'PREP': 4.8046875, 'DET': -...",G,"{'G': 4.48046875, 'PREP': 1.8994140625, 'DET':..."
12,"I can play games, listen to music, and I can d...",PREP,Preposition_Replacing,PREP,"{'G': 2.314453125, 'PREP': 3.546875, 'DET': -4...",PREP,"{'G': 3.251953125, 'PREP': 4.80859375, 'DET': ...",G,"{'G': 4.53515625, 'PREP': 1.8603515625, 'DET':..."
45,"In my opinion, in a hundred of years we'll wea...",PREP,Preposition_Unnecessary,PREP,"{'G': 2.455078125, 'PREP': 3.49609375, 'DET': ...",PREP,"{'G': 3.34765625, 'PREP': 4.80078125, 'DET': -...",G,"{'G': 4.18359375, 'PREP': 2.3359375, 'DET': -6..."
69,"As if that wasn't enough, after the show, I to...",PREP,Preposition_Replacing,PREP,"{'G': 2.66015625, 'PREP': 3.369140625, 'DET': ...",PREP,"{'G': 3.837890625, 'PREP': 4.64453125, 'DET': ...",G,"{'G': 4.9453125, 'PREP': 1.5107421875, 'DET': ..."
99,The reason for this is that I have difficulty ...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.521484375, 'PREP': 3.431640625, 'DET':...",PREP,"{'G': 3.345703125, 'PREP': 4.80078125, 'DET': ...",G,"{'G': 4.94140625, 'PREP': 1.4755859375, 'DET':..."


In [ ]:
list_c = ['v20_l4_e9_preds','O_RoBERTa_preds']
list_w = ['BB_RoBERTa_preds']
baby_and_O_vs_BB = models_compare_labels(all_outfiles, list_c, list_w)
print(baby_and_O_vs_BB.shape)
baby_and_O_vs_BB.head()

(99, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
7,I thought it was a good idea to tell to Pat be...,PREP,Preposition_Unnecessary,PREP,"{'G': 2.671875, 'PREP': 3.353515625, 'DET': -4...",G,"{'G': 4.38671875, 'PREP': 4.2421875, 'DET': -5...",PREP,"{'G': 0.51611328125, 'PREP': 5.640625, 'DET': ..."
292,"Lastly, I would like to learn that what kinds ...",PREP,Preposition_Unnecessary,PREP,"{'G': 2.705078125, 'PREP': 3.302734375, 'DET':...",G,"{'G': 5.03125, 'PREP': 3.068359375, 'DET': -5....",PREP,"{'G': 2.125, 'PREP': 5.265625, 'DET': -6.57812..."
331,"For August, I have already made another appoin...",G,Preposition_Replacing,G,"{'G': 3.16015625, 'PREP': 2.853515625, 'DET': ...",PREP,"{'G': 4.21484375, 'PREP': 4.359375, 'DET': -5....",G,"{'G': 5.53515625, 'PREP': 0.90283203125, 'DET'..."
378,"To begin with, in your advertisement you say t...",G,Preposition_Missing,G,"{'G': 3.0625, 'PREP': 2.966796875, 'DET': -4.6...",PREP,"{'G': 3.603515625, 'PREP': 4.74609375, 'DET': ...",G,"{'G': 4.80078125, 'PREP': 1.6171875, 'DET': -6..."
448,Thank you for organising the school trip to Lo...,G,Preposition_Replacing,G,"{'G': 3.603515625, 'PREP': 2.265625, 'DET': -5...",PREP,"{'G': 3.1953125, 'PREP': 4.79296875, 'DET': -5...",G,"{'G': 5.63671875, 'PREP': 0.75244140625, 'DET'..."


In [ ]:
list_c = ['O_RoBERTa_preds','BB_RoBERTa_preds']
list_w = ['v20_l4_e9_preds']
O_and_BB_vs_baby = models_compare_labels(all_outfiles, list_c, list_w)
print(O_and_BB_vs_baby.shape)
O_and_BB_vs_baby.head()

(165, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
33,Analyses persuades this cousin of Brad to exer...,SVA,irregular_plural_subject_verb_agreement_1,G,"{'G': 4.53125, 'PREP': -2.787109375, 'DET': -4...",SVA,"{'G': 2.814453125, 'PREP': -3.572265625, 'DET'...",SVA,"{'G': 0.0089874267578125, 'PREP': -3.08203125,..."
72,"Firstly, Palace Hotel has been booked for the ...",G,Preposition_Unnecessary,PREP,"{'G': 3.009765625, 'PREP': 3.07421875, 'DET': ...",G,"{'G': 4.55078125, 'PREP': 3.984375, 'DET': -5....",G,"{'G': 5.1953125, 'PREP': 1.287109375, 'DET': -..."
105,A lot of waiters who didn't dislike Katherine ...,G,distractor_agreement_relative_clause,SVA,"{'G': 2.16796875, 'PREP': -2.94140625, 'DET': ...",G,"{'G': 6.7265625, 'PREP': -2.427734375, 'DET': ...",G,"{'G': 7.1796875, 'PREP': -2.447265625, 'DET': ..."
132,"Now, I am writing to you to ask for some detai...",G,Preposition_Missing,PREP,"{'G': 2.732421875, 'PREP': 3.30078125, 'DET': ...",G,"{'G': 4.3984375, 'PREP': 4.19140625, 'DET': -5...",G,"{'G': 5.328125, 'PREP': 1.16015625, 'DET': -6...."
138,The sister of all cashiers force Andrea's best...,SVA,distractor_agreement_relational_noun,G,"{'G': 5.140625, 'PREP': -2.37890625, 'DET': -4...",SVA,"{'G': -0.048095703125, 'PREP': -2.521484375, '...",SVA,"{'G': -2.05078125, 'PREP': -3.12109375, 'DET':..."


In [ ]:
list_c = ['O_RoBERTa_preds','BB_RoBERTa_preds', 'v20_l4_e9_preds']
list_w = []
all_correct = models_compare_labels(all_outfiles, list_c, list_w)
print(all_correct.shape)
all_correct.head()

(5773, 9)


,text,labels,UID,v20_l4_e9_preds,v20_l4_e9_probs,BB_RoBERTa_preds,BB_RoBERTa_probs,O_RoBERTa_preds,O_RoBERTa_probs
0,Those customers examine those commentaries.,G,regular_plural_subject_verb_agreement_1,G,"{'G': 5.1953125, 'PREP': -2.23046875, 'DET': -...",G,"{'G': 6.7890625, 'PREP': -2.380859375, 'DET': ...",G,"{'G': 7.078125, 'PREP': -2.697265625, 'DET': -..."
1,Even Carlos will often happen to die.,G,npi_present_1,G,"{'G': 5.09375, 'PREP': -2.142578125, 'DET': -4...",G,"{'G': 6.71875, 'PREP': -1.349609375, 'DET': -4...",G,"{'G': 7.24609375, 'PREP': -2.16015625, 'DET': ..."
3,Margaret has said Craig discusses himself.,G,principle_A_domain_2,G,"{'G': 5.12109375, 'PREP': -2.26953125, 'DET': ...",G,"{'G': 6.86328125, 'PREP': -2.03125, 'DET': -4....",G,"{'G': 7.203125, 'PREP': -2.46875, 'DET': -5.24..."
4,Julia skated around that popular cafe.,G,determiner_noun_agreement_with_adjective_1,G,"{'G': 4.89453125, 'PREP': -2.4453125, 'DET': -...",G,"{'G': 6.765625, 'PREP': -2.310546875, 'DET': -...",G,"{'G': 7.20703125, 'PREP': -2.451171875, 'DET':..."
6,Many senators scan this black newspaper articles.,DET,determiner_noun_agreement_with_adj_2,DET,"{'G': -2.28515625, 'PREP': -1.166015625, 'DET'...",DET,"{'G': -2.7421875, 'PREP': -2.458984375, 'DET':...",DET,"{'G': -2.662109375, 'PREP': -2.53515625, 'DET'..."


### SAVE

In [ ]:
import os
save_path = '/content/drive/MyDrive/VU Thesis/Code/Error_Analysis/EA_CSV_files'


In [ ]:
# Save all CSV files

o_roberta_vs_bb_roberta.to_csv(os.path.join(save_path,'o_roberta_vs_bb_roberta.csv'), index=False)
bb_roberta_vs_o_roberta.to_csv(os.path.join(save_path,'bb_roberta_vs_o_roberta.csv'), index=False)
bb_roberta_vs_baby.to_csv(os.path.join(save_path,'bb_roberta_vs_baby.csv'), index=False)
baby_vs_bb_roberta.to_csv(os.path.join(save_path,'baby_vs_bb_roberta.csv'), index=False)
baby_vs_o_roberta.to_csv(os.path.join(save_path,'baby_vs_o_roberta.csv'), index=False)
o_roberta_vs_baby.to_csv(os.path.join(save_path,'o_roberta_vs_baby.csv'), index=False)

all_wrongs.to_csv(os.path.join(save_path,'all_wrongs.csv'), index=False)
all_outfiles.to_csv(os.path.join(save_path,'all_outfiles.csv'), index=False)

for key in errors_BB.keys():
    errors_BB[key].to_csv(os.path.join(save_path, f'BB_{key}.csv'), index=False)

for key in errors_O.keys():
    errors_O[key].to_csv(os.path.join(save_path, f'O_{key}.csv'), index=False)

for key in errors.keys():
    errors[key].to_csv(os.path.join(save_path, f'baby_{key}.csv'), index=False)

In [ ]:
baby_vs_O_and_BB.to_csv(os.path.join(save_path,'baby_vs_O_and_BB.csv'), index=False)
BB_vs_baby_and_O.to_csv(os.path.join(save_path,'BB_vs_baby_and_O.csv'), index=False)
O_vs_baby_and_BB.to_csv(os.path.join(save_path,'O_vs_baby_and_BB.csv'), index=False)
baby_and_BB_vs_O.to_csv(os.path.join(save_path,'baby_and_BB_vs_O.csv'), index=False)
baby_and_O_vs_BB.to_csv(os.path.join(save_path,'baby_and_O_vs_BB.csv'), index=False)
O_and_BB_vs_baby.to_csv(os.path.join(save_path,'O_and_BB_vs_baby.csv'), index=False)
all_correct.to_csv(os.path.join(save_path,'all_correct.csv'), index=False)
